Modeling: Same Wavelength
=========================

This script fits a multiple `Imaging` datasets observed at the same wavelength of a galaxy with a
model where:

 - The galaxy's light is a linear parametric `Sersic` bulge and `Exponential` disk.

This script demonstrates how PyAutoGalaxy's multi-dataset modeling tools can also simultaneously analyse datasets
observed at the same wavelength.

An example use case might be analysing undithered HST images before they are combined via the multidrizzing process,
to remove correlated noise in the data.

This is an advanced script and assumes previous knowledge of the core **PyAutoGalaxy** API for galaxy modeling. Thus,
certain parts of code are not documented to ensure the script is concise.

TODO: NEED TO INCLUDE DIFFERENT POINTING / CENTERINGS.

In [ ]:
%matplotlib inline
from pyprojroot import here
workspace_path = str(here())
%cd $workspace_path
print(f"Working Directory has been set to `{workspace_path}`")

from os import path
import autofit as af
import autogalaxy as ag
import autogalaxy.plot as aplt

__Pixel Scales__

If observed at the same wavelength, it is likely the datasets have the same pixel-scale.

Nevertheless, we specify this as a list as there could be an exception.

In [ ]:
pixel_scales_list = [0.1, 0.1]

__Dataset__

Load and plot each multi-wavelength galaxy dataset, using a list of their waveband colors.

In [ ]:
dataset_type = "multi"
dataset_label = "imaging"
dataset_name = "same_wavelength"

dataset_path = path.join("dataset", dataset_type, dataset_label, dataset_name)

dataset_list = [
    ag.Imaging.from_fits(
        data_path=path.join(dataset_path, f"image_{i}.fits"),
        psf_path=path.join(dataset_path, f"psf_{i}.fits"),
        noise_map_path=path.join(dataset_path, f"noise_map_{i}.fits"),
        pixel_scales=pixel_scales,
    )
    for i, pixel_scales in enumerate(pixel_scales_list)
]

for dataset in dataset_list:
    dataset_plotter = aplt.ImagingPlotter(dataset=dataset)
    dataset_plotter.subplot_dataset()

__Mask__

The model-fit requires a `Mask2D` defining the regions of the image we fit the galaxy model to the data, which we define
and use to set up the `Imaging` object that the galaxy model fits.

For multi-wavelength galaxy modeling, we use the same mask for every dataset whenever possible. This is not
absolutely necessary, but provides a more reliable analysis.

In [ ]:
mask_list = [
    ag.Mask2D.circular(
        shape_native=dataset.shape_native, pixel_scales=dataset.pixel_scales, radius=3.0
    )
    for dataset in dataset_list
]


dataset_list = [
    dataset.apply_mask(mask=mask) for imaging, mask in zip(dataset_list, mask_list)
]

for dataset in dataset_list:
    dataset_plotter = aplt.ImagingPlotter(dataset=dataset)
    dataset_plotter.subplot_dataset()

__Analysis__

We create an `Analysis` object for every dataset.

In [ ]:
analysis_list = [ag.AnalysisImaging(dataset=dataset) for dataset in dataset_list]

By summing this list of analysis objects, we create an overall `CombinedAnalysis` which we can use to fit the 
multi-wavelength imaging data, where:

 - The log likelihood function of this summed analysis class is the sum of the log likelihood functions of each 
 individual analysis objects (e.g. the fit to each separate waveband).

 - The summing process ensures that tasks such as outputting results to hard-disk, visualization, etc use a 
 structure that separates each analysis and therefore each dataset.
 
 - Next, we will use this combined analysis to parameterize a model where certain galaxy parameters vary across
 the dataset.

In [ ]:
analysis = sum(analysis_list)

We can parallelize the likelihood function of these analysis classes, whereby each evaluation is performed on a 
different CPU.

In [ ]:
analysis.n_cores = 1

__Model__

We compose our galaxy model using `Model` objects, which represent the galaxies we fit to our data. In this 
example we fit a galaxy model where:

 - The galaxy's bulge is a linear parametric `Sersic` bulge [6 parameters]. 
 
 - The galaxy's disk is a linear parametric `Exponential`disk [5 parameters]. 

The number of free parameters and therefore the dimensionality of non-linear parameter space is N=15.

In [ ]:
bulge = af.Model(ag.lp_linear.Sersic)
disk = af.Model(ag.lp_linear.Exponential)

galaxy = af.Model(ag.Galaxy, redshift=0.5, bulge=bulge, disk=disk)

model = af.Collection(galaxies=af.Collection(galaxy=galaxy))

TODO: Extend code below to vary the dataset centering / pointing across the fit.

In [ ]:
# analysis = analysis.with_free_parameters(model.galaxies.source.bulge.intensity)

__Search__

The model is fitted to the data using a non-linear search. In this example, we use the nested sampling algorithm 
Nautilus (https://nautilus.readthedocs.io/en/latest/).

A full description of the settings below is given in the beginner modeling scripts, if anything is unclear.

In [ ]:
search = af.Nautilus(
    path_prefix=path.join("multi", "modeling"),
    name="same_wavelength",
    unique_tag=dataset_name,
    n_live=100,
    number_of_cores=1,
)

__Model-Fit__

In [ ]:
result_list = search.fit(model=model, analysis=analysis)

__Result__

The result object returned by this model-fit is a list of `Result` objects, because we used a combined analysis.
Each result corresponds to each analysis, and therefore corresponds to the model-fit at that wavelength.

In [ ]:
print(result_list[0].max_log_likelihood_instance)
print(result_list[1].max_log_likelihood_instance)

Plotting each result's galaxies shows that the source appears different, owning to its different intensities.

In [ ]:
for result in result_list:
    galaxies_plotter = aplt.GalaxiesPlotter(
        galaxies=result.max_log_likelihood_galaxies, grid=result.grids.lp
    )
    galaxies_plotter.subplot_galaxies()

    fit_plotter = aplt.FitImagingPlotter(fit=result.max_log_likelihood_fit)
    fit_plotter.subplot_fit()

The `Samples` object still has the dimensions of the overall non-linear search (in this case N=15). 

Therefore, the samples is identical in every result object.

In [ ]:
plotter = aplt.NestPlotter(samples=result_list.samples)
plotter.corner_cornerpy()

Checkout `autogalaxy_workspace/*/imaging/results` for a full description of analysing results in **PyAutoGalaxy**.